In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Cấu hình logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Đường dẫn dataset
DATASET_PATH = "data/UIT-ViHSD_train.csv"

def load_data(dataset_path):
    """Tải dữ liệu từ file CSV."""
    try:
        df = pd.read_csv(dataset_path)
        logger.info(f"Dữ liệu đã được tải thành công từ {dataset_path}")
        return df
    except Exception as e:
        logger.error(f"Lỗi khi tải dữ liệu: {e}")
        raise

def analyze_data(df):
    """Phân tích dữ liệu và hiển thị thông tin."""
    # Hiển thị thông tin tổng quan
    logger.info("\nThông tin tổng quan về dataset:")
    display(df.head())
    logger.info("\nThông tin dataset:")
    print(df.info())
    logger.info("\nThống kê mô tả:")
    print(df.describe())

    # Kiểm tra giá trị null
    logger.info("\nSố lượng giá trị null:")
    null_counts = df.isnull().sum()
    print(null_counts)

    # Vẽ biểu đồ phân phối giá trị null
    if null_counts.sum() > 0:
        plt.figure(figsize=(8, 4))
        sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
        plt.title("Phân phối giá trị null trong dataset")
        plt.show()

    # Phân phối nhãn
    plt.figure(figsize=(6, 4))
    sns.countplot(x=df['label'], palette='Set2')
    plt.title("Phân phối nhãn trong dataset")
    plt.xlabel("Label")
    plt.ylabel("Số lượng")
    plt.show()

    # Phân phối độ dài văn bản
    df['text_length'] = df['text'].apply(len)
    plt.figure(figsize=(8, 5))
    sns.histplot(df['text_length'], bins=50, kde=True, color='blue')
    plt.title("Phân phối độ dài văn bản")
    plt.xlabel("Độ dài văn bản")
    plt.ylabel("Tần suất")
    plt.show()

    # Phân tích từng lớp
    logger.info("\nPhân tích từng lớp:")
    for label in df['label'].unique():
        label_df = df[df['label'] == label]
        logger.info(f"\nLớp {label}:")
        logger.info(f"Số lượng mẫu: {len(label_df)}")
        logger.info(f"Độ dài văn bản trung bình: {label_df['text_length'].mean():.2f}")
        logger.info(f"Độ dài văn bản ngắn nhất: {label_df['text_length'].min()}")
        logger.info(f"Độ dài văn bản dài nhất: {label_df['text_length'].max()}")

    # Hiển thị một số mẫu dữ liệu ngẫu nhiên
    logger.info("\nMẫu dữ liệu ngẫu nhiên:")
    display(df.sample(5))

def main():
    """Hàm chính để thực thi phân tích dữ liệu."""
    try:
        # Tải dữ liệu
        df = load_data(DATASET_PATH)

        # Phân tích dữ liệu
        analyze_data(df)
    except Exception as e:
        logger.error(f"Lỗi trong quá trình phân tích dữ liệu: {e}")

if __name__ == "__main__":
    main()